<a href="https://colab.research.google.com/github/darshit5/Sem-4/blob/master/mainModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ftfy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.1 MB/s eta 0:00:00


In [5]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.4/237.4 kB 4.1 MB/s eta 0:00:00


In [7]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00


In [9]:
import re
from ftfy import fix_text
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords

import os
from docx import Document
import skills_extraction as skills_extraction

In [13]:

# Load dataset:
jd_df=pd.read_csv(r'jd_structured_data.csv')

# Load the extracted resume skills:
file_path=r'/content/CV.pdf'
skills=[]
skills.append(' '.join(word for word in skills_extraction.skills_extractor(file_path)))


In [14]:
def ngrams(string, n=3):
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [15]:
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
tfidf = vectorizer.fit_transform(skills)

nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
jd_test = (jd_df['Processed_JD'].values.astype('U'))

def getNearestN(query):
  queryTFIDF_ = vectorizer.transform(query)
  distances, indices = nbrs.kneighbors(queryTFIDF_)
  return distances, indices

distances, indices = getNearestN(jd_test)
test = list(jd_test)
matches = []

In [16]:
for i,j in enumerate(indices):
    dist=round(distances[i][0],2)

    temp = [dist]
    matches.append(temp)

matches = pd.DataFrame(matches, columns=['Match confidence'])
# Following recommends Top 5 Jobs based on candidate resume:

jd_df['match']=matches['Match confidence']
#jd_df.sort_values('match')
jd_df.sort_values('match', ascending=False).head(5)

,Job Title,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Competitors,Average Salary,Average Revenue,Processed_JD,match
63,Flutter,3.4,CyrusOne\n3.4,"Dallas, TX","Dallas, TX",350.500000,2000,Company - Public,Real Estate,Real Estate,"Digital Realty, CoreSite, Equinix",95.0000,1500.00000,Develop mobile applications and deploy to iOS ...,1.20
103,Flutter,4.0,Peterson Technology Partners\n4.0,"Plano, TX","Park Ridge, IL",350.500000,1997,Contract,Computer Hardware & Software,Information Technology,-1,103.1539,37.50000,"As a Flutter Developer, you will be in charge ...",1.19
901,Data Scientist,3.4,"DatamanUSA, LLC\n3.4","Olympia, WA","Centennial, CO",125.500000,-1,Company - Private,IT Services,Information Technology,-1,89.0000,7.50000,DatamanUSA exciting opportunity Data Scientist...,1.18
345,Data Scientist,-1.0,Seen by Indeed,"Pittsburgh, PA","Austin, TX",3027.393199,-1,Company - Private,-1,-1,-1,103.1539,24319.00076,With one application considered thousands tech...,1.16
214,Data Scientist,-1.0,Seen by Indeed,"Pittsburgh, PA","Austin, TX",3027.393199,-1,Company - Private,-1,-1,-1,103.1539,24319.00076,With one application considered thousands tech...,1.16
